<a href="https://colab.research.google.com/github/satvik94/Embeddings-from-VGGish/blob/master/VGGish_Embeddings_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#VGGish Audio Embedding Colab

This colab demonstrates how to extract the AudioSet embeddings, using a VGGish deep neural network (DNN).

#Importing and Testing the VGGish System

Based on the directions at: https://github.com/tensorflow/models/tree/master/research/audioset

In [1]:
!pip install numpy scipy
!pip install resampy tensorflow six
!pip install tf_slim
!pip install soundfile

     |████████████████████████████████| 512kB 4.8MB/s 
  Found existing installation: tensorflow-estimator 1.15.0
    Uninstalling tensorflow-estimator-1.15.0:
      Successfully uninstalled tensorflow-estimator-1.15.0
     |████████████████████████████████| 122kB 4.9MB/s 


In [2]:
!git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 32913, done.
remote: Total 32913 (delta 0), reused 0 (delta 0), pack-reused 32913
Receiving objects: 100% (32913/32913), 511.79 MiB | 32.34 MiB/s, done.
Resolving deltas: 100% (21099/21099), done.
Checking out files: 100% (2438/2438), done.


In [3]:
# Check to see where are in the kernel's file system.
!pwd

/content


In [4]:
# Grab the VGGish model
!curl -O https://storage.googleapis.com/audioset/vggish_model.ckpt
!curl -O https://storage.googleapis.com/audioset/vggish_pca_params.npz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  277M  100  277M    0     0  56.5M      0  0:00:04  0:00:04 --:--:-- 59.0M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 73020  100 73020    0     0   304k      0 --:--:-- --:--:-- --:--:--  306k


In [5]:
# Make sure we got the model data.
!ls

models	sample_data  vggish_model.ckpt	vggish_pca_params.npz


In [6]:
# Verify the location of the VGGish source files
!ls models/research/audioset/vggish

mel_features.py		  vggish_input.py	 vggish_slim.py
README.md		  vggish_params.py	 vggish_smoke_test.py
vggish_inference_demo.py  vggish_postprocess.py  vggish_train_demo.py


In [0]:
# Copy the source files to the current directory.
!cp models/research/audioset/vggish/* .

In [8]:
# Make sure the source files got copied correctly.
!ls

mel_features.py		  vggish_input.py	 vggish_slim.py
models			  vggish_model.ckpt	 vggish_smoke_test.py
README.md		  vggish_params.py	 vggish_train_demo.py
sample_data		  vggish_pca_params.npz
vggish_inference_demo.py  vggish_postprocess.py


In [9]:
# Run the test, which also loads all the necessary functions.
from vggish_smoke_test import *

W0324 09:13:11.653609 139728207759232 deprecation.py:323] From /tensorflow-1.15.0/python2.7/tensorflow_core/python/compat/v2_compat.py:68: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term



Testing your install of VGGish



W0324 09:13:15.114495 139728207759232 deprecation.py:323] From /tensorflow-1.15.0/python2.7/tensorflow_core/contrib/layers/python/layers/layers.py:1057: apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.


Log Mel Spectrogram example:  [[-4.47297436 -4.29457354 -4.14940631 ... -3.9747003  -3.94774997
  -3.78687669]
 [-4.48589533 -4.28825497 -4.139964   ... -3.98368686 -3.94976505
  -3.7951698 ]
 [-4.46158065 -4.29329706 -4.14905953 ... -3.96442484 -3.94895483
  -3.78619839]
 ...
 [-4.46152626 -4.29365061 -4.14848608 ... -3.96638113 -3.95057575
  -3.78538167]
 [-4.46152595 -4.2936572  -4.14848104 ... -3.96640507 -3.95059567
  -3.78537143]
 [-4.46152565 -4.29366386 -4.14847603 ... -3.96642906 -3.95061564
  -3.78536116]]


W0324 09:13:15.233644 139728207759232 deprecation.py:323] From /tensorflow-1.15.0/python2.7/tensorflow_core/contrib/layers/python/layers/layers.py:1634: flatten (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.flatten instead.


VGGish embedding:  [0.         0.         0.         0.         0.         0.
 0.         0.16137305 0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.8069579
 0.         0.         0.         0.         0.         0.
 0.         0.36792767 0.0358243  0.         0.         0.
 0.         0.38027033 0.13755944 0.9174706  0.8065635  0.
 0.         0.         0.         0.04036267 0.7076243  0.
 0.497839   0.24081804 0.2156543  0.884923   1.1956801  0.6706196
 0.20779456 0.0163987  0.17471854 0.         0.         0.2510081
 0.         0.         0.14607906 0.         0.3988705  0.30542105
 0.12896752 0.         0.         0.         0.         0.
 0.53851354 0.         0.         0.04941072 0.42527407 0.18537286
 0.         0.         0.14753528 0.         0.         0.69933873
 0.45541185 0.05174828 0.         0.01992542 0.         0.
 0.5181578  0.5655761  0.6587975  0.         0.         0.41056335
 0.         0.         0.         0.2576519

#Using the VGGish System

In [0]:
import vggish_slim
import vggish_params
import vggish_input
import soundfile as sf

def CreateVGGishNetwork(hop_size=0.96):   # Hop size is in seconds.
  """Define VGGish model, load the checkpoint, and return a dictionary that points
  to the different tensors defined by the model.
  """
  vggish_slim.define_vggish_slim()
  checkpoint_path = 'vggish_model.ckpt'
  vggish_params.EXAMPLE_HOP_SECONDS = hop_size
  
  vggish_slim.load_vggish_slim_checkpoint(sess, checkpoint_path)

  features_tensor = sess.graph.get_tensor_by_name(
      vggish_params.INPUT_TENSOR_NAME)
  embedding_tensor = sess.graph.get_tensor_by_name(
      vggish_params.OUTPUT_TENSOR_NAME)

  layers = {'conv1': 'vggish/conv1/Relu',
            'pool1': 'vggish/pool1/MaxPool',
            'conv2': 'vggish/conv2/Relu',
            'pool2': 'vggish/pool2/MaxPool',
            'conv3': 'vggish/conv3/conv3_2/Relu',
            'pool3': 'vggish/pool3/MaxPool',
            'conv4': 'vggish/conv4/conv4_2/Relu',
            'pool4': 'vggish/pool4/MaxPool',
            'fc1': 'vggish/fc1/fc1_2/Relu',
            'fc2': 'vggish/fc2/Relu',
            'embedding': 'vggish/embedding',
            'features': 'vggish/input_features',
         }
  g = tf.get_default_graph()
  for k in layers:
    layers[k] = g.get_tensor_by_name( layers[k] + ':0')
    
  return {'features': features_tensor,
          'embedding': embedding_tensor,
          'layers': layers,
         }

In [0]:
# Creating the network
import tensorflow as tf
tf.reset_default_graph()
sess = tf.Session()

vgg = CreateVGGishNetwork(0.96) # The input number represents the duration of the sound file in seconds.


In [0]:
def EmbeddingsFromVGGish(vgg, x, sr):
  '''Run the VGGish model, starting with a sound (x) at sample rate
  (sr). Return a dictionary of embeddings from the different layers
  of the model.'''
  # Produce a batch of log mel spectrogram examples.
  input_batch = vggish_input.waveform_to_examples(x, sr)
  # print('Log Mel Spectrogram example: ', input_batch[0])

  layer_names = vgg['layers'].keys()
  tensors = [vgg['layers'][k] for k in layer_names]
  
  results = sess.run(tensors,
                     feed_dict={vgg['features']: input_batch})

  resdict = {}
  for i, k in enumerate(layer_names):
    resdict[k] = results[i]
    
  return resdict

In [14]:
""" 
Import data into Colab.
Load the wav files as a zip file into PWD. Name the zip file as sounds.
"""
from zipfile import ZipFile
zip_name = "sounds.zip"

with ZipFile(zip_name, 'r') as zip:
  zip.extractall('sounds')
  print("Extracted all sound files into the folder named 'sounds'!!")

Extracted all sound files into the folder named 'sounds'!!


In [15]:
"""
Load all the files into a list.
"""
print("The contents of the 'sounds' folder is ")
!ls sounds/
import glob
sounds = glob.glob('sounds/*.wav')

The contents of the 'sounds' folder is 
acomic.wav  amal.wav


In [16]:
"""
This code cell extracts embeddings from the sound files added to the list. 
"""

# Extract embeddings from first sound file.
print("Extracting embeddings from " + sounds[0])
in_signal, in_sr = sf.read(sounds[0])
resdict = EmbeddingsFromVGGish(vgg, in_signal, in_sr)
em0 = resdict['embedding']
print("The shape of em0 is" + str(em0.shape))
em = np.copy(em0)

# Extract embeddings from remaining files.
for s in sounds[1:]:
  print("Extracting embeddings from " + s)
  in_signal, in_sr = sf.read(s)
  resdict = EmbeddingsFromVGGish(vgg, in_signal, in_sr)
  em_s = resdict['embedding']
  print("The shape of em_s is" + str(em_s.shape))
  em = np.concatenate((em, em_s), axis = 0)
  print("The shape of em is" + str(em.shape))

Extracting embeddings from sounds/amal.wav
The shape of em0 is(31, 128)
Extracting embeddings from sounds/acomic.wav
The shape of em_s is(31, 128)
The shape of em is(62, 128)


In [0]:
"""
Store the numpy matrices.
"""
np.save("embeddings.npy", em)


In [18]:
"""
Load the numpy arrays
"""
em_load = np.load("embeddings.npy")
print("em_load shape is {}".format(em_load.shape))

np.array_equal(em, em_load)

em_load shape is (62, 128)


True

In [0]:
for k in resdict:
  print k, resdict[k].shape

fc1 (31, 4096)
fc2 (31, 128)
features (31, 96, 64)
conv3 (31, 24, 16, 256)
conv2 (31, 48, 32, 128)
conv1 (31, 96, 64, 64)
embedding (31, 128)
conv4 (31, 12, 8, 512)
pool3 (31, 12, 8, 256)
pool2 (31, 24, 16, 128)
pool1 (31, 48, 32, 64)
pool4 (31, 6, 4, 512)
